In [ ]:
import matplotlib.pyplot as pp
import numpy as np

In [ ]:
pp.style.use('ggplot')
np.random.seed(42)

In [ ]:
def gaussian_process_kernel(x_1, x_2, length=1.0, sigma=1.0):
    distance_squared = np.sum(x_1**2, 1).reshape(-1, 1) + np.sum(x_2**2, 1) - 2 * np.dot(x_1, x_2.T)
    return sigma**2 * np.exp(-distance_squared / length**2 / 2)

def gaussian_process_prior_sample(n, x):
    mean = np.zeros(x.shape)
    covariance = gaussian_process_kernel(x, x)
    return np.random.multivariate_normal(mean.ravel(), covariance, n).T

def gaussian_process_posterior_predictive(x_s, x, y, sigma=0):
    k = gaussian_process_kernel(x, x)
    k_s = gaussian_process_kernel(x_s, x)
    k_ss = gaussian_process_kernel(x_s, x_s)
    k_inv = np.linalg.inv(k + sigma**2 * np.eye(x.shape[0]))
    mean_s = k_s.dot(k_inv).dot(y)
    covariance_s = k_ss - k_s.dot(k_inv).dot(k_s.T)
    return mean_s, covariance_s

def gaussian_process_posterior_predictive_sample(n, x_s, x, y, **options):
    mean, covariance = gaussian_process_posterior_predictive(x_s, x, y, **options)
    return np.random.multivariate_normal(mean.ravel(), covariance, n).T

def noise_prior_sample(n, x, tau=1):
    weight = np.random.normal(0, tau, (x.shape[1] + 1, n))
    sigma = np.sqrt(np.exp(weight[0, :] + x.dot(weight[1:, :])))
    return np.random.normal(0, sigma)

In [ ]:
x = np.arange(-5, 5, 0.1).reshape(-1, 1)
y = gaussian_process_prior_sample(3, x)

pp.plot(x, y)

In [ ]:
y = noise_prior_sample(3, x)

pp.plot(x, y)

In [ ]:
x_train = np.array([-4, -3, -2, -1, 1]).reshape(-1, 1)
y_train = np.sin(x_train)
y = gaussian_process_posterior_predictive_sample(3, x, x_train, y_train)

pp.plot(x, y)
pp.scatter(x_train, y_train)